In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
#!pip install scikit-learn
#pip install numpy==1.21
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers trl accelerate torch bitsandbytes peft datasets -qU


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
import tensorflow as tf
from tensorflow import keras
from transformers import TrainingArguments
import re


In [5]:
data = []
for filename in os.listdir('new_dataset/'):
    if filename.endswith("(smallchunks).csv"):
        data.append(filename)
dfs = []
for i in data:
  df = pd.read_csv(f'new_dataset/{i}', engine='python', encoding='utf-8', on_bad_lines='skip')

  df_list = df.iloc[:, 0].tolist()
  for i in df_list:
    dfs.append(i)
res_df = pd.DataFrame(dfs)
res_df = res_df.rename(columns={0: 'texts'})
res_df.dropna(inplace=True)

In [7]:
prompts_train, prompts_test = train_test_split(res_df, test_size=0.1)

prompts_train = pd.DataFrame(prompts_train)
prompts_test = pd.DataFrame(prompts_test)
prompts_train.to_csv("train_mini1.csv", index=False, header=False)
prompts_test.to_csv("test_mini1.csv", index=False, header=False)

train_df = pd.read_csv('train_mini1.csv')
test_df = pd.read_csv('test_mini1.csv')

train_df.columns = ['texts']
test_df.columns = ['texts']

train_ds = Dataset.from_pandas(train_df, split="train")
test_ds = Dataset.from_pandas(test_df, split="test")

In [ ]:
dataset = DatasetDict({
    "train": train_ds,
    "test": test_ds
})

def tokenize_function(examples):
    # extract text
    text = examples["texts"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = dataset.map(tokenize_function, batched=True)
#tokenized_data = dataset.map(create_prompt, batched=True)

In [ ]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [12]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [ ]:
before = generate_response('### Instruction:\nImagine you are a famous writer. Use the provided input to create a piece of fictional text that would be as nice as original text.### Input:\nHarry Potter meets a spider.\n\n### Original text:', model)

In [ ]:
before.replace('\n', '')

In [13]:
##train

peft_config = LoraConfig(
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    lora_dropout=0.1,
    r=25,
    bias="none",
    task_type="TEXT_GENERATION"
)

In [14]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
args = TrainingArguments(
  output_dir = "mistral_fiction_generation4",
  num_train_epochs=6,
  #max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 4,
  warmup_steps = 0,
  logging_steps=10,
  save_strategy="epoch",
  evaluation_strategy="epoch",
  #evaluation_strategy="steps",
  #eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant_with_warmup',
)

In [19]:
def create_prompt(sample):
  bos_token = "<s>"
  original_system_message = "You are a famous writer and you are writing a book.\n\nYou must write only the plot of THIS CHAPTER and nothing else: \n SETTING:"
  system_message = "Imagine you are a famous writer. Use the provided input to create a piece of fictional text that would be as nice as original text."
  #response = sample["texts"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
  response = re.search(r'(?<=\[\/INST\]).*', sample["texts"], re.DOTALL).group().strip() if re.search(r'(?<=\[\/INST\]).*', sample["texts"], re.DOTALL) else None
  input = re.search(r'(?<=\[INST\]).*(?=\[\/INST\])', sample["texts"], re.DOTALL).group().strip() if re.search(r'(?<=\[INST\]).*(?=\[\/INST\])', sample["texts"], re.DOTALL) else None
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += "### Instruction:"
  full_prompt += "\n" + system_message
  full_prompt += "\n\n### Input:"
  full_prompt += "\n" + input
  full_prompt += "\n\n### Original text:"
  full_prompt += "\n" + response
  full_prompt += eos_token

  to_delete = "You are a famous writer and you are writing a book.\n\nYou must write only the plot of THIS CHAPTER and nothing else: \n SETTING:"
  full_prompt = full_prompt.replace(to_delete, "")

  return full_prompt

In [ ]:
first_row_prompt = create_prompt(prompts_train.iloc[0])
print(first_row_prompt)

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=False,
  formatting_func=create_prompt,
  args=args,
  train_dataset=tokenized_data["train"],
  eval_dataset=tokenized_data["test"]
)


In [ ]:
train_result = trainer.train()

In [ ]:
metrics = train_result.metrics
max_train_samples = training_args.max_train_samples if training_args.max_train_samples is not None else len(train_dataset)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

In [188]:
trainer.save_model("mistral_fiction4")

In [ ]:
merged_model = model.merge_and_unload()

In [171]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [190]:
generate_response("### Instruction:\nImagine you are a famous writer. Use the provided input to create a piece of fictional text that would be as nice as original text.### Input:\nHarry Potter meets a spider.\n\n### Original text:", merged_model)

'<s> ### Instruction:\nImagine you are a famous writer. Use the provided input to create a piece of fictional text that would be as nice as original text.### Input:\nHarry Potter meets a spider.\n\n### Original text: \nHarry Potter sits down in the Gryffindor common room. After a while, he observes a spider crawling along the ceiling. He knows he isn\'t supposed to disturb it but he can\'t help it. He whispers to the spider, "You\'ll have to forgive me, but spiders give me the creeps." The spider doesn\'t seem to notice him and continues to crawl along the ceiling. Harry lets out a sigh and stands up, "This is ridiculous." He turns to his friends, "Guys, can you get a broom or something and get rid of that spider?" The spider then drops from the ceiling and falls on Harry\'s head.         \n### Fictional text:\nHarry Potter spent his time in the Gryffindor common room. After a while, he watched a spider crawl along the ceiling. Harry knew it was one of the unbreakable rules in his educ